In [1]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>
# To horizontal scroll dataframe table

In [2]:
import graphframes
from pyspark.sql import SparkSession
from graphframes import GraphFrame
import os
from pyspark.sql.functions import col
from pyspark.sql.functions import exp
from pyspark.sql.functions import sum as _sum # avoid same name with sum from python
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import min, max

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
b3760b54-4731-46bc-8305-7197ac2a92a7,spark-187243d6957a472a880a37bf8f28649c,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Function to calculating weight of the edge(txn) by the age of the txn
def logistic(x,
             a=2,
             b=1 / 64):
    return a / (1.0 + np.exp(-b * x))
def normalize(df):
    normalized_df=(df-df.min())/(df.max()-df.min())
    return normalized_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession \
    .builder \
    .config("fs.s3a.access.key", "***") \
    .config("fs.s3a.secret.key", "***") \
    .appName("Test") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 1. Data Preprocessing

In [28]:
#''' # get all file

bucket="s3a://octan-labs-bsc/"
path="/transactions/"
data_df=spark.read.format("csv") \
    .option("header", True) \
    .load(bucket+path+'*/*/*.csv')

''' test one file

bucket="s3a://octan-labs-bsc/"
path="/transactions/start_block=0000420000/"
data_df=spark.read.format("csv") \
    .option("header", True) \
    .load(bucket+path+'*/*.csv')
    
#'''

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

' test one file\nbucket="s3a://octan-labs-bsc/"\npath="/transactions/start_block=0000420000/"\ndata_df=spark.read.format("csv")     .option("header", True)     .load(bucket+path+\'*/*.csv\')\n#'

In [6]:
data_df = data_df.select(col('block_number'),col('from_address'),col('to_address'),col('gas'),col('gas_price'))
data_df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------------------------------------+------------------------------------------+-------------------+------------+
|block_number|from_address                              |to_address                                |gas                |gas_price   |
+------------+------------------------------------------+------------------------------------------+-------------------+------------+
|420200      |0xfc5eb925da6e7d830adf3fce080206db073be569|0xae9269f27437f0fcbc232d39ec814844a51d6b8f|64004              |20000000000 |
|420200      |0xfbb684d91129c07b318212fbbfb6ac00644c1ce3|0xae9269f27437f0fcbc232d39ec814844a51d6b8f|74004              |20000000000 |
|420200      |0x7ae2f5b9e386cd1b50a4550696d957cb4900f03a|0x0000000000000000000000000000000000001002|9223372036854775807|0           |
|420200      |0x7ae2f5b9e386cd1b50a4550696d957cb4900f03a|0x0000000000000000000000000000000000001000|9223372036854775807|0           |
|420202      |0xd1ddc0f727935f376ba6f9f51b1e0fb481c25fb1|0x1ff

#### Summary before processing

In [29]:
print('Total: ',data_df.count())
block_min = data_df.select(min('block_number')).first()[0]
block_max = data_df.select(max('block_number')).first()[0]
print('first block:',block_min,'last block:',block_max)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total:  234853
first block: 1 last block: 99879

#### Get the Used gas

In [30]:
data_df = data_df.withColumn('gas_used (bnb)', col('gas') * col('gas_price'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
data_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+------------+-----------------+--------------------+--------------------+--------------------+-------------------+-----------+--------------------+---------------+---------------+------------------------+----------------+--------------+
|                hash|nonce|          block_hash|block_number|transaction_index|        from_address|          to_address|               value|                gas|  gas_price|               input|block_timestamp|max_fee_per_gas|max_priority_fee_per_gas|transaction_type|gas_used (bnb)|
+--------------------+-----+--------------------+------------+-----------------+--------------------+--------------------+--------------------+-------------------+-----------+--------------------+---------------+---------------+------------------------+----------------+--------------+
|0xe30271d29e02c04...|   36|0x534c7cc400e6d79...|      400302|                0|0x5d0d8aa6ee5d444...|0x75ca8f6c82df5fd...|                   0

### remove loop, keep last txn, remove nan and null address

In [32]:
 # remove loop
data_df = data_df.filter(col('from_address') != col('to_address'))
 # keep last txn
window_spec = Window.partitionBy(['from_address', 'to_address']).orderBy(col('block_number').desc())
data_df = data_df.withColumn('row_num', row_number().over(window_spec)).filter(col('row_num') == 1).drop('row_num')
# remove nan
data_df = data_df.dropna(subset=['from_address'])
data_df = data_df.dropna(subset=['to_address'])
# remove null address (address for burning)
data_df = data_df.filter(col('from_address') != '0x0000000000000000000000000000000000000000') 
data_df = data_df.filter(col('from_address') != '0x000000000000000000000000000000000000dEaD') 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Example:
Before pre-processing

| block_num | from_address | to_address |
| --- | --- | --- |
| 1 | a | b |
| 1 | a | b |
| 2 | a | b |
| 2 | a | null |
| 3 | a |  |
| 3 | a | a |

After pre-processing

| block_num | from_address | to_address |
| --- | --- | --- |
| 2 | a | b |


#### Summary after pre-processing

In [33]:
print('Total:', data_df.count(), 'txns')
block_min = data_df.select(min('block_number')).first()[0]
block_max = data_df.select(max('block_number')).first()[0]
print('first block:',block_min,'last block:',block_max)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total: 19154 txns
first block: 1 last block: 99879

## 2. Calculting weight for node and edge

In this verion, graphframe will be used to calculate page rank. However, due to limitation of this package, it cannot assign weight to edges and node. </br>
Therefore, graphframe part will be used as a place holder for better package. </br>
Weight for node and edge are still calculated but will not be used.

In [34]:
activate_time=90
blockperday_set=28800
data_df = data_df.withColumn('block_date', ((col('block_number') - block_min - activate_time) / blockperday_set).cast('int'))
data_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+------------+-----------------+--------------------+--------------------+--------------------+------+------------+--------------------+---------------+---------------+------------------------+----------------+--------------+----------+
|                hash|nonce|          block_hash|block_number|transaction_index|        from_address|          to_address|               value|   gas|   gas_price|               input|block_timestamp|max_fee_per_gas|max_priority_fee_per_gas|transaction_type|gas_used (bnb)|block_date|
+--------------------+-----+--------------------+------------+-----------------+--------------------+--------------------+--------------------+------+------------+--------------------+---------------+---------------+------------------------+----------------+--------------+----------+
|0xfc5827f6567c2b4...|   14|0x4e2dde1c55a7235...|      402026|                0|0x000000004fa9e63...|0x8f76c99c3e96371...|  100000000000000000| 2

In [35]:
a = 1
b = 1/64
data_df = data_df.withColumn('age_weight', a / (1.0 + exp(-b * data_df['block_date'])))
data_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+------------+-----------------+--------------------+--------------------+--------------------+------+------------+--------------------+---------------+---------------+------------------------+----------------+--------------+----------+------------------+
|                hash|nonce|          block_hash|block_number|transaction_index|        from_address|          to_address|               value|   gas|   gas_price|               input|block_timestamp|max_fee_per_gas|max_priority_fee_per_gas|transaction_type|gas_used (bnb)|block_date|        age_weight|
+--------------------+-----+--------------------+------------+-----------------+--------------------+--------------------+--------------------+------+------------+--------------------+---------------+---------------+------------------------+----------------+--------------+----------+------------------+
|0xfc5827f6567c2b4...|   14|0x4e2dde1c55a7235...|      402026|                0|0x000000

### Get weight for edge

In [36]:
to_weights_df = data_df.select(['to_address', 'age_weight'])
node_age_weights = to_weights_df.groupBy('to_address').agg(_sum('age_weight').alias('age_weight'))
s_weights = node_age_weights.agg(_sum('age_weight')).collect()[0][0] # result is a list of row so need [0][0]
personalized_weights = {row['to_address']: row['age_weight']/s_weights for row in node_age_weights.collect()}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
data_df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------------+-----+------------------------------------------------------------------+------------+-----------------+------------------------------------------+------------------------------------------+----------------------+------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+---------------+------------------------+----------------+--------------+----------+------------------+
|hash                                                              |nonce|block_hash                                    

### Get weight for node(vertices)

In [38]:
vertices_df = data_df.select(col("from_address").alias("id")).union(data_df.select(col("to_address").alias("id")))
vertices_df = vertices_df.distinct()
vertices_df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------+
|id                                        |
+------------------------------------------+
|0x4f129fbd6fe4c632c04998adfb392d93da412ce1|
|0x5558bf3dc2f5b0c658bbbefe0b2d8d3507efeb69|
|0x56e5272a583d3b490062f69e183d4a510d007ef8|
|0x58932ae7796b4861790d49199c992f1fca3d5609|
|0xc8b9bbba82cbe9f56ea20344cd5dc84066f994ed|
|0xd576b5994865c94a0a6449687c2a08e47fd77bb4|
|0x1c537fcf80c1069716ea7a151100edbd55b99a4c|
|0x4e8f1cf9a9ddcadee3340dada2cba5508d340b4e|
|0x54b4e4a37808b8d71a07f494ec20cfab80e2827d|
|0x555a56c3f71d8d1c411bd3b10434385c411547fe|
|0x67909f0fd4d76342b85108412d5c97ed5b33c49f|
|0x70ddb0f995d235e150e50800b751dfd4379bd5cd|
|0x83003d1b9d4e47296ee1ca6667af26c8e8e3d4da|
|0xc0d9ae4426209cde6b3488edc0b63f6cdae525d6|
|0xd0e1b043cb7423ce3d419f50d6c554eba0f4ca13|
|0xe8af1475e115859259ae5347029c70a017957d71|
|0xecb0a819c0d991405f2bb4f97a3ef7b3b77dd2ab|
|0xf764cc532f61789b0cf56dab37348f02afc70887|
|0xfa97c22a03d8522988c709c24283c0918a59c795|
|0x0cae1e6

In [39]:
weight_df = spark.createDataFrame(
    personalized_weights.items(),
    ["id", "weight"]
)
result_df = vertices_df.join(weight_df, "id", "left").select("id", "weight")
vertices_df_with_personalized_weight = result_df.withColumnRenamed("weight", "personalized weight")
vertices_df_with_personalized_weight=vertices_df_with_personalized_weight.fillna(0)
vertices_df_with_personalized_weight.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------+---------------------+
|id                                        |personalized weight  |
+------------------------------------------+---------------------+
|0xae9269f27437f0fcbc232d39ec814844a51d6b8f|0.0819356513237302   |
|0x56e5272a583d3b490062f69e183d4a510d007ef8|5.201713004851768E-5 |
|0x555a56c3f71d8d1c411bd3b10434385c411547fe|1.0439921588543996E-4|
|0x9e2f1fdcf8ec30d3d4fc83534f1244d6e1bc9e15|5.238208583692229E-5 |
|0x1558c9987bceaec8b2868f9b5e7bade7c4ac0dee|4.8711786094065975E-5|
|0x1c537fcf80c1069716ea7a151100edbd55b99a4c|1.0476417167384458E-4|
|0xe8af1475e115859259ae5347029c70a017957d71|5.0184557612870767E-5|
|0xf764cc532f61789b0cf56dab37348f02afc70887|5.238208583692229E-5 |
|0xfb3b893159fd02e6300a37ae0e8789163a635160|5.238208583692229E-5 |
|0xf89dda78a4e5e17d28fb14d7075955af6229150d|4.9816791220551864E-5|
|0xfa97c22a03d8522988c709c24283c0918a59c795|5.238208583692229E-5 |
|0x0cae1e6c5e8b4109097e9e900e9f9554cc26b66c|5.238208583692229E

In [40]:
edges_df = data_df.select(col("from_address").alias("src"), col("to_address").alias("dst"), col("age_weight").alias("weight"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
print(vertices_df.count(),edges_df.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4924 19154

### Get transaction, degree and gas information

In [42]:
# calcualte totall gas, total transfer, total receive

total_gas_used_df = data_df.groupBy('from_address').agg(_sum('gas_used (bnb)').alias('Total gas'))
total_gas_used_df=total_gas_used_df.withColumnRenamed('from_address', 'Address')

total_transfer_df = data_df.groupBy('from_address').count().withColumnRenamed('count', 'Total Transfers')
total_transfer_df=total_transfer_df.withColumnRenamed('from_address', 'Address')

total_receive_df = data_df.groupBy('to_address').count().withColumnRenamed('count', 'Total Receives')
total_receive_df=total_receive_df.withColumnRenamed('to_address', 'Address')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Intiate the Graph

Currently using graphframe, which will **not** support age weight and node weight

In [43]:
graph = GraphFrame(vertices_df, edges_df)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
# count nodes and edges
num_vertices = graph.vertices.count()
num_edges = graph.edges.count()

print("Number of nodes:", num_vertices)
print("Number of edges:", num_edges)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of nodes: 4924
Number of edges: 19154

In [45]:
# calculate in , out degree
in_degrees = graph.inDegrees
in_degrees = in_degrees.withColumnRenamed('id', 'Address')
out_degrees = graph.outDegrees
out_degrees = out_degrees.withColumnRenamed('id', 'Address')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Page Rank Calculation

In [46]:
results = graph.pageRank(resetProbability=0.15, tol=0.01)
#results = graph.pageRank(resetProbability=0.5, maxIter= 10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
result_df=results.vertices.select("id", "pagerank").toDF("Address", "Reputation Score")
result_df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------+------------------+
|Address                                   |Reputation Score  |
+------------------------------------------+------------------+
|0xae9269f27437f0fcbc232d39ec814844a51d6b8f|171.10100331631136|
|0x56e5272a583d3b490062f69e183d4a510d007ef8|0.6083179071295058|
|0x555a56c3f71d8d1c411bd3b10434385c411547fe|1.2765580580132707|
|0x9e2f1fdcf8ec30d3d4fc83534f1244d6e1bc9e15|0.7251332310642303|
|0x1558c9987bceaec8b2868f9b5e7bade7c4ac0dee|0.671171661408802 |
|0x1c537fcf80c1069716ea7a151100edbd55b99a4c|0.647421047266197 |
|0xe8af1475e115859259ae5347029c70a017957d71|0.6083179071295058|
|0xf764cc532f61789b0cf56dab37348f02afc70887|0.6083179071295058|
|0xfb3b893159fd02e6300a37ae0e8789163a635160|0.6083179071295058|
|0xf89dda78a4e5e17d28fb14d7075955af6229150d|0.6343580843183398|
|0xfa97c22a03d8522988c709c24283c0918a59c795|0.8522184365084768|
|0x15222740c04c23a356244b10d20e94d8ca6b63d5|0.6083179071295058|
|0x64598aab661f730396db2e3f430b5a20904a1

In [48]:
# calculate in , out degree
in_degrees = graph.inDegrees
in_degrees = in_degrees.withColumnRenamed('id', 'Address')
out_degrees = graph.outDegrees
out_degrees = out_degrees.withColumnRenamed('id', 'Address')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Merging txn , gas, and degree information to the page rank result

In [49]:
merged_df = result_df.join(total_gas_used_df, 'Address', 'left')
merged_df = merged_df.join(total_transfer_df, 'Address', 'left')
merged_df = merged_df.join(total_receive_df, 'Address', 'left')
merged_df = merged_df.join(in_degrees, on='Address', how='left').join(out_degrees, on='Address', how='left')
merged_df = merged_df.withColumn('total_degree', merged_df['inDegree'] + merged_df['outDegree'])
merged_df = merged_df.fillna(0)
merged_df = merged_df.replace(float('-inf'), 0)
merged_df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------+------------------+-------------+---------------+--------------+--------+---------+------------+
|Address                                   |Reputation Score  |Total gas    |Total Transfers|Total Receives|inDegree|outDegree|total_degree|
+------------------------------------------+------------------+-------------+---------------+--------------+--------+---------+------------+
|0x56e5272a583d3b490062f69e183d4a510d007ef8|0.6083179071295058|3.76712664E16|8              |1             |1       |8        |9           |
|0xb0dd17567933fc80810d7fd6b39409eb03dd23a6|1.6754423377397183|7.543875E15  |2              |2             |2       |2        |4           |
|0xc8b9bbba82cbe9f56ea20344cd5dc84066f994ed|0.6083179071295058|3.435702E16  |7              |1             |1       |7        |8           |
|0xd576b5994865c94a0a6449687c2a08e47fd77bb4|0.6083179071295058|2.28181252E17|38             |1             |1       |38       |39          |
|0xae9269f274